In [3]:
import pandas as pd

# COVID 19 - Data Preperation

In this notebook, I outline how I prepared data for the Tableau dashboard

The data was gathered the [Github Repository](https://github.com/CSSEGISandData/COVID-19) created by the Johns Hopkins University Center for Systems Science and Engineering. 

In [4]:
# Read datasets from CSSE github repo
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# Getting Ready for Tableau


In [5]:
# Changing to data to long format
confirmed = pd.melt(confirmed,id_vars = confirmed.columns[:4], value_vars = confirmed.columns[4:], var_name = 'Date', value_name = 'Confirmed')
deaths = pd.melt(deaths, id_vars = deaths.columns[:4], value_vars = deaths.columns[4:], var_name = 'Date', value_name = 'Deaths')
recoveries = pd.melt(recoveries, id_vars = recoveries.columns[:4], value_vars = recoveries.columns[4:], var_name = 'Date', value_name = 'Recoveries' )

In [6]:
#confirmed.head(5)
#deaths.head(5)
#recoveries.head(5)

,Province/State,Country/Region,Lat,Long,Date,Recoveries
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [8]:
#confirmed.tail(5)
#deaths.tail(5)
#recoveries.tail(5)

,Province/State,Country/Region,Lat,Long,Date,Confirmed
230035,NaN,West Bank and Gaza,31.952200,35.233200,4/10/22,656287
230036,NaN,Winter Olympics 2022,39.904200,116.407400,4/10/22,535
230037,NaN,Yemen,15.552727,48.516388,4/10/22,11813
230038,NaN,Zambia,-13.133897,27.849332,4/10/22,317483
230039,NaN,Zimbabwe,-19.015438,29.154857,4/10/22,246958


A cursory analysis reveals that the data for some of the countries has been reported based on region.

In [267]:
confirmed['Country/Region'][~confirmed['Province/State'].isnull()].unique()


array(['Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands',
       'New Zealand', 'United Kingdom'], dtype=object)

In [268]:
deaths['Country/Region'][~deaths['Province/State'].isnull()].unique()

array(['Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands',
       'New Zealand', 'United Kingdom'], dtype=object)

In [269]:
recoveries['Country/Region'][~recoveries['Province/State'].isnull()].unique()

array(['Australia', 'China', 'Denmark', 'France', 'Netherlands',
       'New Zealand', 'United Kingdom'], dtype=object)

As seen above, the countries that are divided based on region are: 
'Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands', 'New Zealand', 'United Kingdom'. 

However, the recoveries dataset does not divide Canada by region, instead reporting on recoveries in the entire country.

To keep the Tableau visualisations consistent, all observations with Regions were merged into one.

In [7]:
# mec stands for 'multiple entry countries'
mec = ['Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands', 'New Zealand', 'United Kingdom']

# Creating a dataframe without any of the muliple entry countries
# Combines the confirmed, deaths, and recoveries into 1 table
covid_df = (confirmed[~confirmed['Country/Region'].isin(mec)]).reset_index(drop=True)
covid_df['Deaths'] = (deaths[~deaths['Country/Region'].isin(mec)].iloc[:,-1]).reset_index(drop=True)
covid_df['Recoveries'] = (recoveries[~recoveries['Country/Region'].isin(mec)].iloc[:,-1]).reset_index(drop=True)
del covid_df['Province/State']


To add the data from the removed countries (mec) back into the dataframe, it must be consistent in format with the other countries.

In [9]:
#Creating an empty list
mec_df = []

for country in mec:
    #Creating the sum of each statistic based on the date 
    confirmed_total = confirmed[confirmed['Country/Region'] == country].groupby('Date').sum()[['Confirmed']]
    deaths_total = deaths[deaths['Country/Region'] == country].groupby('Date').sum()[['Deaths']]
    recoveries_total = recoveries[recoveries['Country/Region'] == country].groupby('Date').sum()[['Recoveries']]

    template_1 = confirmed[confirmed['Country/Region'] == country]
    # Each country should only have 1 distinct date for 1 observation 
    template_1 = template_1.drop_duplicates(subset = 'Date', keep = 'first')
    del template_1['Province/State']
    template_2 = template_1[template_1.columns[:-1]].reset_index(drop=True)
    
    # Merging the confirmed, death, and recoveries table into 1.
    template_2 = template_2.merge(confirmed_total,how='inner',left_on='Date',right_index=True)
    template_2 = template_2.merge(deaths_total,how='inner',left_on='Date',right_index=True)
    template_2 = template_2.merge(recoveries_total,how='inner',left_on='Date',right_index=True)

    #Appends a small dataframe into the list.
    mec_df.append(template_2)

# Creates a large dataframe from all the smaller dataframes in the list.
mec_df = pd.concat(mec_df, ignore_index=True)
mec_df

,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recoveries
0,Australia,-35.4735,149.0124,1/22/20,0,0,0
1,Australia,-35.4735,149.0124,1/23/20,0,0,0
2,Australia,-35.4735,149.0124,1/24/20,0,0,0
3,Australia,-35.4735,149.0124,1/25/20,0,0,0
4,Australia,-35.4735,149.0124,1/26/20,4,0,0
...,...,...,...,...,...,...,...
6475,United Kingdom,18.2206,-63.0686,4/6/22,21625530,169698,0
6476,United Kingdom,18.2206,-63.0686,4/7/22,21674196,170020,0
6477,United Kingdom,18.2206,-63.0686,4/8/22,21716180,170367,0
6478,United Kingdom,18.2206,-63.0686,4/9/22,21716180,170367,0


In [272]:

covid_df = (pd.concat([covid_df,mec_df])).reset_index(drop=True)

covid_df['Date'] = pd.to_datetime(covid_df.Date)

covid_df.sort_values(['Date', 'Country/Region'])

,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recoveries
0,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0
1,Albania,41.153300,20.168300,2020-01-22,0,0,0
2,Algeria,28.033900,1.659600,2020-01-22,0,0,0
3,Andorra,42.506300,1.521800,2020-01-22,0,0,0
4,Angola,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...
151995,West Bank and Gaza,31.952200,35.233200,2022-03-31,655750,5655,0
151996,Winter Olympics 2022,39.904200,116.407400,2022-03-31,535,0,0
151997,Yemen,15.552727,48.516388,2022-03-31,11806,2143,0
151998,Zambia,-13.133897,27.849332,2022-03-31,316850,3966,0


# Verifying Countries
When running the tail function, I noticed that Winter Olympics 2022 was an observation. As this is not a country, I decided to reference it with another data set.

A [Kaggle Dataset](https://www.kaggle.com/tanuprabhu/population-by-country-2020) provides a list of countries from [Worldometer](https://www.worldometers.info/world-population/population-by-country/)

In [273]:
pop = pd.read_csv('/Users/ishanamin/Desktop/Projects/Tableau Dashboard/2020population.csv')

pop.head(5)

,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39%,5540090,153,9388211,-348399.0,1.7,38,61%,18.47%
1,India,1382345085,0.99%,13586631,464,2973190,-532687.0,2.2,28,35%,17.70%
2,United States,331341050,0.59%,1937734,36,9147420,954806.0,1.8,38,83%,4.25%
3,Indonesia,274021604,1.07%,2898047,151,1811570,-98955.0,2.3,30,56%,3.51%
4,Pakistan,221612785,2.00%,4327022,287,770880,-233379.0,3.6,23,35%,2.83%


In [274]:

countries = covid_df['Country/Region'].unique()

# returns countries from covid_df that are not in pop
for country in countries:
    if country not in pop['Country'].unique():
        print (country)

Antarctica
Burma
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Czechia
Diamond Princess
Korea, South
Kosovo
MS Zaandam
Saint Kitts and Nevis
Saint Vincent and the Grenadines
Sao Tome and Principe
Summer Olympics 2020
Taiwan*
US
West Bank and Gaza
Winter Olympics 2022


John Hopkins recorded data from Cruise ships (MS Zaandam, Diamond Princess) and the olympics. I removed these observations from the dataframe.

In [275]:
not_countries = ['Diamond Princess', 'MS Zaandam', 'Summer Olympics 2020', 'Winter Olympics 2022']
covid_df = covid_df[~covid_df['Country/Region'].isin(not_countries)]

There are some inconsistency with the John Hopkins dataset and International Naming conventions.

In [276]:
country_mapper = {
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Democratic Republic of Congo',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Czechia': 'Czech Republic (Czechia)',
    'Korea, South': 'South Korea',
    'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
    'Taiwan*': 'Taiwan',
    'US': 'United States',
    'West Bank and Gaza': 'Israel',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis',
    'Burma': 'Myanmar',
    'Sao Tome and Principe': 'Sao Tome & Principe'
}

covid_df['Country/Region'] = covid_df['Country/Region'].replace(country_mapper)
covid_df.index = covid_df['Country/Region']

In [277]:
covid_df.to_csv('COVID-19.csv')